# Old Babylonian Lists of Trees and Wooden Objects

## Introduction to research questions and analysis plan

We are interested in understanding relationships among extant versions of lexical texts. Patterns in similarity of these texts may provide important information about text provenance and/or routes of influence from one geographical area onto another. 

We are also interested in understanding the patterns by which lexical texts evolved and changed. 

In comparing versions of a lexical text we may think of four types of features: 

1) presence or absence of entries  
2) order of entries within a section  
3) order of sections in a document  
4) spelling of words  

The following sections will explore these four features independently and in combination to uncover patterns of similarity among documents.

## Introduction to dataset and data structure

This notebook uses data from the Digital Corpus of Cuneiform Lexical Texts ([DCCLT](http://oracc.org/dcclt)) derived from parsed JSON files. For the JSON output from the Open Richly Annotated Cuneiform Corpus ([ORACC](http://oracc.org)) see the [ORACC Open Data documentation](http://oracc.museum.upenn.edu/doc/opendata/index.html).  

The JSON files are parsed with the notebook [grab_json.ipynb](https://github.com/ErinBecker/digital-humanities-phylogenetics/blob/master/scripts/grab_json.ipynb). This notebook takes an input file, identifying the text IDs of the documents to be parsed. The input file is [ob_lists_wood.txt](https://github.com/ErinBecker/digital-humanities-phylogenetics/blob/master/data/text_ids/ob_lists_wood.txt). 

The input file lists all the Text IDs of Old Babylonian lists of trees and wooden objects currently in [DCCLT](http://oracc.org/dcclt), as well as the composite text of the [Nippur version](http://oracc.org/Q000039). Text IDs consist of a P plus a six-digit number (commonly referred to as P-number) that is recognized by [ORACC](http://oracc.org) and by the Cuneiform Digital Library Initiative ([CDLI](http://cdli.ucla.edu)) and that has become the de-facto standard in Assyriology. [CDLI](http://cdli.ucla.edu) provides metadata (provenience, period, publication, museum number, etc) for each text. Composite text IDs consist of a Q plus a six-digit number (for instance Q000039). Texts that have not (yet) been cataloged in [CDLI](http://cdli.ucla.edu) receive a (temporary) six-digit X number.

The data are placed in the directory [data](https://github.com/ErinBecker/digital-humanities-phylogenetics/tree/master/data). The are comma-separated files have the following fields: 

| field         | description                     |
|-----------	|------------------------------------------------------------------------------------------------------------------------------------------------------	|
| id_line   	| consists of a text ID (P, Q, or X number) plus a reference number 	|
| label 	| line number: obverse/reverse, column number, line number (e.g. o ii 16')                                                          	|
| lemma      	| Sumerian words in lemmatized form (e.g. lugal[king]N); for unlemmatized words the raw transliteration is taken                                                                                  	|
| base      	| Sumerian words in original spelling, but without morphological prefixes or suffixes   |
| extent | (for missing data): how many lines or columns (restricted vocabulary) are missing|
| scope | (for missing data): what is missing - line, column, face, or surface (restricted vocabulary) |

There are various types of missing data, represented in different ways. A word that is present, but not lemmatized is represented in its transliterated form, followed by [NA]NA (that is: Guideword and POS are both NA). Words that are partly or entirely illegible on the original document are by definition unlemmatized and are handled the same way.

Lines or multiple lines that are missing are indicated in the fields `extent` and `scope`. `Extent` gives the number of missing lines (or missing columns, etc). The restricted vocabulary includes numbers and the words 'n' (unknown), 'beginning', and 'rest'. `Scope` indicates the scope of the missing text: line, column, obverse, etc.

| type         | how represented                     |
|-----------	|------------------------------------------------------------------------------------------------------------------------------------------------------	|
| words with unknown lemmatization| siki-siki[NA]NA |
| illegible words | x[NA]NA |
| known number of missing lines 	|extent: '5' scope: 'line' |
| unknown number of missing lines	|extent: 'n' scope: 'line |
| two missing columns  | extent: '2' scope: 'column'|
  


In [1]:
import pandas as pd
import numpy as np
import re

# Reading in and structuring the data
Open file `ob_lists_wood.csv` and create a Dataframe in Pandas.  

In [2]:
file = '../data/ob_lists_wood.csv'
df = pd.read_csv(file).drop('Unnamed: 0', axis=1)

In [3]:
df.head()

,id_line,label,lemma,base,extent,scope
0,P117395.2,o 1,ŋešed[key]N,{ŋeš}e₃-a,NaN,NaN
1,P117395.3,o 2,pakud[~tree]N,{ŋeš}pa-kud,NaN,NaN
2,P117395.4,o 3,raba[clamp]N,{ŋeš}raba,NaN,NaN
3,P117404.2,o 1,ig[door]N eren[cedar]N,{ŋeš}ig {ŋeš}eren,NaN,NaN
4,P117404.3,o 2,ig[door]N dib[board]N,{ŋeš}ig dib,NaN,NaN


## id_text and line
The variable `id_line` contains the text ID plus a reference. The reference may be to a column, a line, or a set of broken lines. The text ID is put in the variable `id_text` and the reference is turned into an integer and put in the variable `line`. The variables `id_text` and `line` are then used to sort the dataframe.

In [4]:
df['id_text'] = df['id_line'].str[:7]
df['line'] = [int(re.sub('.+\.', '', line)) for line in df['id_line']] #create a line number for sorting
df = df.sort_values(['id_text', 'line']).reset_index(drop=True)
df.head()

,id_line,label,lemma,base,extent,scope,id_text,line
0,P117395.2,o 1,ŋešed[key]N,{ŋeš}e₃-a,NaN,NaN,P117395,2
1,P117395.3,o 2,pakud[~tree]N,{ŋeš}pa-kud,NaN,NaN,P117395,3
2,P117395.4,o 3,raba[clamp]N,{ŋeš}raba,NaN,NaN,P117395,4
3,P117404.2,o 1,ig[door]N eren[cedar]N,{ŋeš}ig {ŋeš}eren,NaN,NaN,P117404,2
4,P117404.3,o 2,ig[door]N dib[board]N,{ŋeš}ig dib,NaN,NaN,P117404,3


## The `skip` variable

The variable `skip` is used to compute the distance between two lines in the data set. If a line has data (in `label`, `lemma`, and `base`) `skip` = 0. If, however, the original text has 5 missing lines, there will be a separate row, where `skip` is 5. If there is a gap in the text of unknown length, `skip` will be NaN.

The `skip` variable works as follows (simplified data representation):

|`label` | `lemma` | `skip` | `line`
|--------|----------|--------|------|
| o ii 4 | gigir[chariot]N    | 0| 43 |
| o ii 5 | sahargi[dustguard]N gigir[chariot]N | 0| 44 |
| NaN     | NAN     | 5 | 45 |
| o ii 11 | margida[wagon]N | 0| 46 |


The distance between the `margida[wagon]N` line and the `gigir[chariot]N` line is 7 (`line`₂ - `line`₁ + `skip`₁:₂ -1).

The variable `skip` is computed from the [ORACC](http://oracc.org) variables `extent` and `scope`, which are part of the so-called \$-line conventions. These conventions are explained in more detail [here](http://oracc.org/doc/help/editinginatf/primer/structuretutorial). A 'strict' \$-line uses a limited vocabulary to describe the preservation or state of the object on which the text is written. Examples of strict \$-lines are:,
* \$ beginning of column missing
* \$ 7 lines traces

In these examples '7' and 'beginning' are the `extent`; 'column' and 'line' are `scope` ('missing' and 'traces' are `state`. The variable 'state' is ignored here - treating 'missing', 'broken', 'traces', etc. all as absence of data).

If lines are missing the `extent` variable will indicate the number of missing lines or columns. A line with data has `extent` NaN.

The variable `skip` is computed as follows:

* if the line has data (in `label`, `lemma`, and `base`) `skip` = 0
* if `scope` == 'column', or anything other than 'line', `skip` = NaN
* if `extent` is a digit, `skip` is the integer version of that digit
* if `extent` is 'n' or 'beginning' (or anything other than a digit), `skip` = NaN

Because NaN cannot be used in a column of integers, `skip` will become a float.

## Note to Erin

I am proposing here to introduce a new variable `skip` (rather than adjust the variable `extent`). I think that is clearer to outsiders (and to our future selves). I hope the explanation of how it is done is also clearer. I haven't actually done what I describe here - because it would mess with your datastructure. Essentially, everything remains exactly the same, but what was called `extent` will now be called `skip`. Note that the formula for line distance needs more thought. The number of rows that have some `skip` value needs to be subtracted - there should be some smart way of doing that.

We could, of course, use `999` instead of `NaN` for `skip` - but I prefer to use `NaN` here and to present the change to `999` as a feature of handing the data off to R.

In [5]:
df.extent = df.extent.fillna('0')
df['skip'] = [int(n) if n.isdigit() else np.NaN for n in df.extent]

The variable `scope` may include 'line', 'column', 'obverse', etc. Only if scope is `line` the variable 'extent' is meaningful (if, say, 2 columns are missing, 'extent' is '2' but should be NaN because we do not know how many lines those 2 columns represent). If `scope` is NaN `extent` is '0' and should remain so. After this operation the column 'scope' can be dropped.

In [6]:
df.skip = [df.skip[i] if df.scope[i] in ['line', np.NaN] else np.NaN for i in range(len(df))] # line to be activated when introducing `skip`
df = df.drop(['scope', 'extent'], axis=1)
df.head()

,id_line,label,lemma,base,id_text,line,skip
0,P117395.2,o 1,ŋešed[key]N,{ŋeš}e₃-a,P117395,2,0.0
1,P117395.3,o 2,pakud[~tree]N,{ŋeš}pa-kud,P117395,3,0.0
2,P117395.4,o 3,raba[clamp]N,{ŋeš}raba,P117395,4,0.0
3,P117404.2,o 1,ig[door]N eren[cedar]N,{ŋeš}ig {ŋeš}eren,P117404,2,0.0
4,P117404.3,o 2,ig[door]N dib[board]N,{ŋeš}ig dib,P117404,3,0.0


## Create Expressions
A line in a lexical text may contain more than one word. Usually a list is divided into sections by keyword, for instance:

| text                	| translation                      	|
|---------------------	|----------------------------------	|
| {ŋeš}gigir          	| chariot                          	|
| {ŋeš}e₂ gigir       	| chariot cabin                    	|
| {ŋeš}e₂ usan₃ gigir 	| storage box for the chariot whip 	|
| {ŋeš}gaba gigir     	| breastwork of a chariot          	|

In the comparison between different versions of the list the individual words are less interesting than the *entries*, that is: the sequence of words in a single line. In order to look at entries (rather than words), words in an entry are connected by underscores (_).

In [7]:
df['entry'] = df['lemma']
df['entry'] = df['entry'].str.replace(' ', '_')
df.head()

,id_line,label,lemma,base,id_text,line,skip,entry
0,P117395.2,o 1,ŋešed[key]N,{ŋeš}e₃-a,P117395,2,0.0,ŋešed[key]N
1,P117395.3,o 2,pakud[~tree]N,{ŋeš}pa-kud,P117395,3,0.0,pakud[~tree]N
2,P117395.4,o 3,raba[clamp]N,{ŋeš}raba,P117395,4,0.0,raba[clamp]N
3,P117404.2,o 1,ig[door]N eren[cedar]N,{ŋeš}ig {ŋeš}eren,P117404,2,0.0,ig[door]N_eren[cedar]N
4,P117404.3,o 2,ig[door]N dib[board]N,{ŋeš}ig dib,P117404,3,0.0,ig[door]N_dib[board]N


### Passing data to R

The first step in passing our data from Python to R is converting the Python null value (`NaN`) to the R null value (`NA`). We do this by converting all `NaN` values in character/string columns to the intermediate value `unknown` and all `NaN` values in our numeric `skip` column to the intermediate value `999`.

We will then convert both of these values to `NA` after passing the data to R.

The directory `data/pass` is used for passing data from Python to R and vv.

In [8]:
df.skip = df.skip.fillna(999)
df = df.fillna('unknown')
df.to_csv("../data/pass/ob_lists_wood_w_id_text.csv")